In [1]:
from importlib import reload
import watermarking.watermark_processor

watermarking.watermark_processor = reload(watermarking.watermark_processor)
from watermarking.watermark_processor import RepetitionPenaltyLogitsProcessor
from transformers import LogitsProcessorList, MinLengthLogitsProcessor, LogitsProcessor
from watermarking.utils.text_tools import truncate
from watermarking.utils.load_local import load_local_model_or_tokenizer
from datasets import load_dataset, load_from_disk

/home/jianghaoyu/anaconda3/envs/kgw/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("facebook/opt-1.3b")
model = AutoModelForCausalLM.from_pretrained("facebook/opt-1.3b")
# tokenizer = load_local_model_or_tokenizer("facebook/opt-1.3b", 'tokenizer')
# model = load_local_model_or_tokenizer("facebook/opt-1.3b", 'model')

[proxychains] Strict chain  ...  127.0.0.1:8890  ...  127.0.0.1:8890  ...  huggingface.co:443  ...  OK
[proxychains] Strict chain  ...  127.0.0.1:8890  ...  127.0.0.1:8890  ...  huggingface.co:443  ...  OK


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
lm_tokenizer = AutoTokenizer.from_pretrained('gpt2')
lm_model = AutoModelForCausalLM.from_pretrained('gpt2').to('cuda:0')

In [3]:
model = model.to('cuda:1')

In [4]:
c4_sliced_and_filted = load_from_disk('./c4-train.00000-of-00512_sliced')
c4_sliced_and_filted = c4_sliced_and_filted['train'].shuffle(seed=42).select(
    range(100))

In [5]:
# select a prompt

sample_idx = 98
input_text = c4_sliced_and_filted[sample_idx]['text']
tokenized_input = tokenizer(input_text, return_tensors='pt').to(model.device)
tokenized_input = truncate(tokenized_input, max_length=300)
# # print(tokenized_input)
# input_ids = tokenized_input['input_ids'].cpu().tolist()
# print(len(input_ids[0]))
# print(input_ids[0])
# tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
# print(tokens)

In [6]:
min_length_processor = MinLengthLogitsProcessor(min_length=1000,
                                                eos_token_id=tokenizer.eos_token_id)
repetition_processor = RepetitionPenaltyLogitsProcessor(penalty=1.5)

In [7]:
from importlib import reload
import watermarking.watermark_processors.message_models.lm_message_model
watermarking.watermark_processors.message_models.lm_message_model = reload(watermarking.watermark_processors.message_models.lm_message_model)
import watermarking.watermark_processors.message_model_processor
watermarking.watermark_processors.message_model_processor = reload(watermarking.watermark_processors.message_model_processor)


from watermarking.watermark_processors.message_models.lm_message_model import LMMessageModel
from watermarking.watermark_processors.message_model_processor import WmProcessorMessageModel

In [8]:
lm_message_model = LMMessageModel(tokenizer=tokenizer,lm_model=model,lm_tokenizer=tokenizer,
    delta = 1.1, lm_prefix_len=10, lm_topk=-1, message_code_len = 20,random_permutation_num=50)
wm_precessor_message_model = WmProcessorMessageModel(message_model=lm_message_model,tokenizer=tokenizer,
    encode_ratio=5,max_confidence_lbd=0.5,strategy='max_confidence', message=[42,54])

In [9]:
start_length = tokenized_input['input_ids'].shape[-1]
wm_precessor_message_model.start_length = start_length
output_tokens = model.generate(**tokenized_input, max_new_tokens=200, num_beams=4,
                               logits_processor=LogitsProcessorList(
                                   [min_length_processor, repetition_processor,
                                    wm_precessor_message_model]))

output_text = tokenizer.decode(output_tokens[0][tokenized_input['input_ids'].shape[-1]:],
                               skip_special_tokens=True)
print(output_text)
prefix_and_output_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
# print(prefix_and_output_text)

/home/jianghaoyu/CTWL/watermarking/watermark_processors/message_model_processor.py:183: UserWarning: The reduce argument of torch.scatter with Tensor src is deprecated and will be removed in a future PyTorch release. Use torch.scatter_reduce instead for more reduction options. (Triggered internally at ../aten/src/ATen/native/TensorAdvancedIndexing.cpp:231.)
  scores.scatter_(1, topk_indices, log_Ps, reduce='add')


 express their views during the primary elections.
‘‘He (Chiyangwa) does not listen to us. He does not know what we want," he said.
Dinha also accused Chiyangwa of favouring his close allies in the primary elections.
The Zvimba South constituency is one of the strongholds of President Emmerson Mnangagwa.
Chiyangwa, however, dismissed the allegations, saying they were aimed at tarnishing his image.
‘‘I am confident that I will win this election. I am very sure of my victory,’’ he said.
He added: ‘‘I have been in politics for more than 20 years. I have never seen anything like this in my life.
‘‘This is just another attempt by the opposition to tarnish my image.''
Chiyangwa said he would work hard to improve the lives of ordinary Zimbabweans


In [10]:
log_probs = wm_precessor_message_model.decode(output_text)
print(log_probs)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:01<00:00, 10.59it/s]


([42, 454698], (tensor([[1.1000, 0.0000, 1.1000,  ..., 0.0000, 1.1000, 1.1000],
        [0.0000, 1.1000, 1.1000,  ..., 0.0000, 1.1000, 0.0000],
        [1.1000, 1.1000, 1.1000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [1.1000, 1.1000, 1.1000,  ..., 1.1000, 1.1000, 1.1000],
        [1.1000, 0.0000, 1.1000,  ..., 1.1000, 0.0000, 1.1000],
        [1.1000, 1.1000, 1.1000,  ..., 1.1000, 0.0000, 1.1000]]), [(92, 2, 0.3561956286430359), (79, 0, 0.05510837957262993)]))


In [29]:
log_probs[0]

[42, 454698]

In [21]:
log_probs

([42],
 (tensor([[1.1000, 0.0000, 1.1000,  ..., 0.0000, 1.1000, 1.1000],
          [0.0000, 1.1000, 1.1000,  ..., 0.0000, 1.1000, 0.0000],
          [1.1000, 1.1000, 1.1000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [1.1000, 1.1000, 1.1000,  ..., 1.1000, 1.1000, 1.1000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 1.1000, 1.1000],
          [1.1000, 1.1000, 1.1000,  ..., 1.1000, 1.1000, 1.1000]]),
  [(169, 5, 0.9507791996002197)]))

In [6]:
import watermarking.watermark_processors.random_processor
watermarking.watermark_processors.random_processor = reload(watermarking.watermark_processors.random_processor)

from watermarking.watermark_processors.random_processor import WmProcessorRandom
random_processor = WmProcessorRandom(message=[42,5,22,5465,34], tokenizer=tokenizer,delta=1.5,message_code_len = 20,
     top_k=100, encode_ratio = 5.
    )

In [7]:
random_processor.start_length = tokenized_input['input_ids'].shape[-1]
t_output_tokens = model.generate(**tokenized_input, max_new_tokens=200, num_beams=4,
                               logits_processor=LogitsProcessorList(
                                   [min_length_processor, repetition_processor,
                                    random_processor]))
t_output_text = tokenizer.decode(t_output_tokens[0][tokenized_input['input_ids'].shape[-1]:],
                               skip_special_tokens=True)
t_prefix_and_output_text = tokenizer.decode(t_output_tokens[0], skip_special_tokens=True)

NameError: name 'tokenized_input' is not defined

In [45]:
t_log_probs = random_processor.decode(t_output_text)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 2264.05it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 800.00 MiB. GPU 

In [33]:
t_log_probs[0]

[42, 5]

In [34]:
no_wm_output_tokens = model.generate(**tokenized_input, max_new_tokens=200, num_beams=4,
                                     logits_processor=LogitsProcessorList(
                                       [min_length_processor, repetition_processor]))
no_wm_output_text = tokenizer.decode(no_wm_output_tokens[0][tokenized_input['input_ids'].shape[-1]:],
                                     skip_special_tokens=True)
no_wm_prefix_and_output_text = tokenizer.decode(no_wm_output_tokens[0], skip_special_tokens=True)

In [ ]:
oracle_tokenizer = AutoTokenizer.from_pretrained("facebook/opt-2.7b")
oracle_model = AutoModelForCausalLM.from_pretrained("facebook/opt-2.7b")

# oracle_tokenizer = load_local_model_or_tokenizer('facebook/opt-2.7b', 'tokenizer')
# oracle_model = load_local_model_or_tokenizer('facebook/opt-2.7b', 'model')
oracle_model = oracle_model.to('cuda:0')

/home/jianghaoyu/anaconda3/envs/kgw/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
[proxychains] Strict chain  ...  127.0.0.1:8890  ...  127.0.0.1:8890  ...  cdn-lfs.huggingface.co:443  ...  OK


In [22]:
from watermarking.experiments.watermark import compute_ppl_single

loss, ppl = compute_ppl_single(prefix_and_output_text=prefix_and_output_text,
                               oracle_model_name='facebook/opt-2.7b',
                               output_text=output_text,
                               oracle_model=oracle_model, oracle_tokenizer=oracle_tokenizer)
print(loss, ppl)

0.9879757165908813 2.6857922077178955


In [23]:
from watermarking.experiments.watermark import compute_ppl_single

loss, ppl = compute_ppl_single(prefix_and_output_text=t_prefix_and_output_text,
                               oracle_model_name='facebook/opt-2.7b',
                               output_text=t_output_text,
                               oracle_model=oracle_model, oracle_tokenizer=oracle_tokenizer)
print(loss, ppl)

1.0407838821411133 2.8314356803894043


In [29]:
loss, ppl = compute_ppl_single(prefix_and_output_text=no_wm_prefix_and_output_text,
                               oracle_model_name='facebook/opt-2.7b',
                               output_text=no_wm_output_text,
                               oracle_model=oracle_model, oracle_tokenizer=oracle_tokenizer)
print(loss, ppl)

0.8734689950942993 2.395205497741699
